<a href="https://colab.research.google.com/github/Djjdjdjdjjdjdjd/roop/blob/main/roop_v1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##一张图片，一个视频，一键换脸！

[![打开](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dream80/roop_colab/blob/main/roop_v1_3.ipynb)

基于[roop](https://github.com/s0md3v/roop)项目的一个脚本你，可以进行图片换脸，也可以视频换脸。可以自定义参数。  

特点：就是快捷方便，没有其他。  

如果要高质量换脸还是推荐DeepFaclab[教程](https://www.deepfaker.xyz/)

脚本仅供学习研究，请勿用于任何非法用途！

使用教程：https://www.tonyisstark.com/1240.html  
离线版本：https://www.tonyisstark.com/1311.html

公众号：托尼不是塔克  
博客：[tonyisstark.com](https://www.tonyisstark.com/)

In [1]:
#@title 1.克隆源代码
!git clone https://github.com/s0md3v/roop.git
!git clone https://github.com/dream80/roop_colab.git


Cloning into 'roop'...
remote: Enumerating objects: 1540, done.
remote: Total 1540 (delta 0), reused 0 (delta 0), pack-reused 1540 (from 1)
Receiving objects: 100% (1540/1540), 97.45 MiB | 15.28 MiB/s, done.
Resolving deltas: 100% (904/904), done.
Cloning into 'roop_colab'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 141 (delta 80), reused 52 (delta 46), pack-reused 32 (from 1)
Receiving objects: 100% (141/141), 15.06 MiB | 16.37 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [2]:
#@title 2.进入项目
%cd /content/roop

Version = "v1.3.2" #@param ["v1.3.0","v1.3.1","v1.3.2"]

vdict = {
    "v1.3.0": "142284916a5448acdf1a7fdca56d01c5aecafdad",
    "v1.3.1": "e0642702f388ea16089a1d4a929728fa69457d60",
    "v1.3.2": "c2d1feb17a9c51061b52cae5897136528f3b80cc",
}

commit=vdict.get(Version, "Invalid Version")

vcmd= f"checkout {commit}"


!git $vcmd

/content/roop
Note: switching to 'c2d1feb17a9c51061b52cae5897136528f3b80cc'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at c2d1feb Next (#810)


In [1]:
#@title 3.安装依赖
#有红色警告不用担心！
%cd /content/roop
!pip install -r requirements.txt
!pip install onnxruntime-gpu==1.17.0 --index-url=https://pkgs.dev.azure.com/onnxruntime/onnxruntime/_packaging/onnxruntime-cuda-12/pypi/simple
#!pip install onnxruntime-gpu==1.15.0

/content/roop
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Ignoring tkinterdnd2-universal: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
Ignoring onnxruntime: markers 'python_version != "3.9" and sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring onnxruntime-coreml: markers 'python_version == "3.9" and sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring onnxruntime-silicon: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
Looking in indexes: https://pkgs.dev.azure.com/onnxruntime/onnxruntime/_packaging/onnxruntime-cuda-12/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 MB 6.7 MB/s eta 0:00:00
  Attempting uninstall: onnxruntime-gpu
    Found existing installation: onnxruntime-gpu 1.15.1
    Uninstalling onnxruntime-gpu-1.15.1:
      Successful

In [ ]:
#@title 4.准备素材
source = "/content/roop_colab/1.jpg" #@param {type:"string"}
target = "/content/roop_colab/2.mp4" #@param {type:"string"}
output = "/content/roop_colab/out.mp4" #@param {type:"string"}

source : 源素素材路径，包含人脸的图片  

target : 目标图片或者视频路径，也要包含人脸  

output : 输出路径，如果targe是图片，这里也改成图片后缀  


In [ ]:
#@title 5.开始换脸

%cd /content/roop

Device = "cuda" #@param ["cuda", "cpu"]

Processor = "face_swapper face_enhancer" #@param ["face_swapper face_enhancer", "face_swapper","face_enhancer"]

VideoEncoder = "libx264" #@param ['libx264', 'libx265', 'libvpx-vp9', 'h264_nvenc', 'hevc_nvenc']

VideoQuality = "35" #@param {type:"string"}

TempFormat = "jpg" #@param ["jpg", "png"]

TempQuality = "0" #@param {type:"string"}


KeepFPS = True #@param {type:"boolean"}
SkipAudio = False #@param {type:"boolean"}
KeepFrames = False #@param {type:"boolean"}
ManyFaces = False #@param {type:"boolean"}


KeepFPS ="--keep-fps" if KeepFPS==True else ""
SkipAudio ="--skip-audio" if SkipAudio==True else ""
KeepFrames ="--keep-frames" if KeepFrames==True else ""
ManyFaces ="--many-faces" if ManyFaces==True else ""



cmd = f"run.py --execution-provider {Device} -s {source} -t {target} -o {output} --frame-processor {Processor} --output-video-encoder {VideoEncoder} --output-video-quality {VideoQuality} {KeepFPS} {SkipAudio} {KeepFrames} {ManyFaces} --temp-frame-format {TempFormat} --temp-frame-quality {TempQuality}"
print("cmd:"+cmd)


!wget https://github.com/dream80/roop_colab/releases/download/v0.0.1/inswapper_128.onnx
!mkdir models
!mv inswapper_128.onnx models/

!wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth
!mv GFPGANv1.4.pth models/

!python $cmd


Device : 选CUDA就是使用GPU，选CPU就用CPU，如果你GPU配额用完了，可以用CPU顶一下。或者开Pro  

Precessor : 选择换脸还是人脸修复，默认全部启用，也可以单独使用一个功能。  

VideoEncoder : 视频编码器选择，一般用libx264。  

VideoQuality : 视频质量，默认35，取值范围0-100  

TempFormat ： 缓存帧格式，可以使jpg或者png  

TempQuality ： 视频质量，默认0，取值范围0-100  

KeepFPS : 保持帧率不变  

SkipAudio ： 丢弃音轨   

KeepFrames ： 保留分帧后的图片  

ManyFaces ： 替换所有人脸

#补充说明

  



## 视频文件在哪里？  

换脸成功后的视频保存在roop_colab文件里面  


## 如何使用自己的素材？  

如果要替换图片和视频，只需要在左边的文件管理里面右键上传，然后改一下名字就可以。了。  
素材文件都在roop_colab文件中   
1.jpg：换脸图片  
2.mp4：换脸视频  
out.mp4：是换脸后的视频  

点击左侧文件，找到MP4,右键可以下载！



## 解除NSFW封印

不多说，提示都在这里了  
MAX_PROBABILITY = 1



## 大文件处理

如果文件比较大，直接在Colab上传会有难度，可以先传到谷歌云盘。然后挂载云盘就可以了，修改一下第五步里面图片和视频的地址就可以了。挂载谷歌盘之后也可以把输出文件直接指向云盘。

如何需要使用谷歌云盘的文件，可以点击这个脚本，挂载云盘。

from google.colab import drive  
drive.mount('/content/drive')